In [1]:
# importações 
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px 
import plotly.offline as py 
import plotly.graph_objs as go
import pandas as pd 
from dash.dependencies import Input, Output

#Manuseio de dados 
df = pd.read_csv('olist_classified_public_dataset.csv') #leitura do csv realizada pelo pandas, adicionda a um dataframe 'df'
df = df.dropna(thresh=33) # Retirada do not a number da coluna 33--> classificação
df.order_purchase_timestamp = pd.to_datetime(df.order_purchase_timestamp) # Mudança do tipo, de object para datetime 
df['tempo de venda'] = df.order_purchase_timestamp.dt.to_period('M').astype(str) # Mudança para ano-mes 
aux_data = df.values.tolist() # Mudança do dataframe para uma matrix/lista

In [2]:
'''Grafico Barras'''

# Relação mês_valor
tempo = []          # lista vazia para adição do tempo 
vendas_periodo = [] # Lista vazia para relação do tempo com o valor
valor = 3           # Variável valor recebendo 3 para melhor comprensão do que se trata a coluna "valores"
mes_ano = 34        # Variável mes_ano recebendo 34 para melhor comprensão do que se trata a coluna "mes_ano" 
qtd = 5             # coluna do numero de itens
        
for linha in range(len(aux_data)):                              # Laço que irá percorrer a aux_data  
    vendas = (aux_data[linha][valor])*(aux_data[linha][qtd])    # vendas irá receber o valor do produto de acordo com a leitura da linha valor e quantidade 
    periodo = aux_data[linha][mes_ano]                          # periodo irá receber a tempo de venda de acordo com a leitura da linha 
    vendas_periodo.append([vendas,periodo])                     # agregando valor e com o periodo, criando assim a relação vendas_perido
    if (vendas_periodo[linha][1] not in tempo):                 # Se o periodo de vendas dado ainda nn estiver adicionado na lista tempo,ele será agregado
        tempo.append(vendas_periodo[linha][1])  

tempo.sort()  #colocando tempo em ordem
preço = []    #lista vazia para agreção do preço 
soma = 0      #contador inicia com 0

for cont in range(len(tempo)):                      # Cont ira pecorrer a quantidade do tempo
    for linha in range(len(vendas_periodo)):        # linha ira percorrer a lista vendas_periodo                                              
        if (vendas_periodo[linha][1]==tempo[cont]): # Conferir se o tempo do periodo é corresponde ao tempo crescente dos meses
            soma+=vendas_periodo[linha][0]          # somando os valores que satisfazem a condição acima
    preço.append(soma)                              # Adicionando a soma na lista preço                         
    soma=0                                          # Zerando o contador   

''' Plotagem do gráfico '''      

data0 = [go.Bar(x=tempo,                             # data0 Armazena informações a respeito do gráfico, bem tipo "go.bar"                           
                y=preço,                             # A correspondenciado dos eixos, "x=tempo" e "y=tempo"
                marker = {'color': '#13D6C0',        # maker=recebe caracterísca de layout da barra, sendo color sua cor, line sendo linha, com sua respectivas
                          'line': {'color': 'black', # caracteristicas como cor e espessura "width"
                                   'width': 3}
                        },
              )
       ]

# Criando Layout                                                    
configuracoes_layout = go.Layout(title='Vendas no Periodo',             # Definição da legenda das linhas como título
                                 yaxis={'title':'Valores em Vendas'},   # Definição da legenda do eixo y e x    
                                 xaxis={'title':'Periodo'})

# Objeto figura

fig0 = go.Figure(data=data0, layout=configuracoes_layout)  # Armazenamento da figura e de seus rectivos dados em fig0         
fig0.update_layout(plot_bgcolor="#242423")                 # Definição de fundo 
fig0.update_layout(paper_bgcolor="#242423")  
fig0.update_layout(font_color="white")   

In [3]:
#vendas_periodo[0:10]

In [4]:
#tempo

In [5]:
#df.head()

In [6]:
''' Gráfico categorias '''

# Relacionando categorias por valor
categorias_faturamento=[]                                     # Criação lista categoria por faturamento
name = 14                                                     # Classificando variavel name com 14 "respectiva coluna com os nomes de produtos"
for linha in range (len(aux_data)):                           # Percorrendo aux_data   
    categorias=aux_data[linha][name]                          # Selecionando os nomes das categorias
    faturamento=aux_data[linha][valor]*aux_data[linha][qtd]   # Selecionando valor*qnt de cada categoria
    categorias_faturamento.append([categorias,faturamento])   # Relacionando cada categoria com seu faturamento faturamento 

#Laço que inclui apenas as categorias dentro da lista
categorias = []                                                     # Criando lista categoria
produto = 0                                                         # Classificando variavel produto com 0 "respectiva coluna com os nomes de produtos"
for linha in range (len(categorias_faturamento)):                   # Percorrendo as linhas de categorias_faturamento  
    if categorias_faturamento [linha][produto] not in categorias:   # Condição para inclusão na lista categoria, ainda nn estar na mesma        
        categorias.append(categorias_faturamento[linha][produto])   # Adição definitiva a lista    

# Laço que soma valor de cada categoria por mês
faturamento_total_por_categoria=[]                                    # Criando lista faturamento_total_por_categoria                                        
soma=0                                                                # Iniciando contador do 0
for linha in range(len(categorias)):                                  # Percorrer numero de linhas de categorias  
    for x in range (len(categorias_faturamento)):                     # Percorrendo numero de linhas de categorias_faturamento
        if (categorias_faturamento[x][0]==categorias[linha]):         # Condição pra soma do faturamento_total_por_categoria, terem a mesma categoria
            soma+=categorias_faturamento[x][1]                        # Adição a lista soma o valor da categoria satisfeita acima      
    faturamento_total_por_categoria.append(soma)                      # Adição da soma a lista faturamento_total_por_categoria
    soma=0                                                            # zerando soma                                             

trace1 = go.Bar(y=faturamento_total_por_categoria,   # trace1 Armazena informações a respeito do gráfico, bem tipo "go.bar"
                x=categorias,                        # A correspondenciado dos eixos, "x=categorias" e "y=faturamento_total_por_categoria"    
                marker = {'color': '#13D6C0',        # maker=recebe caracterísca de layout da barra, sendo color sua cor, line sendo linha, com sua respectivas
                          'line': {'color': 'black', # Caracteristicas como cor e espessura "width"
                                   'width': 3}
                        }         
              )

data1 = [trace1]   # Armazenando dados do trace1 em data1 

# Criando Layout
configuracoes_layout = go.Layout(title='Vendas por categoria de Produtos',             # Definição da legenda das linhas como título            
                   xaxis=dict(titlefont=dict(size=50,color='black'),tickangle=50),     # Definição da legenda do eixo y e x
                   yaxis={'title': 'Faturamento'},
                   
                   )

# Objeto figura

fig1 = go.Figure(data=data1, layout=configuracoes_layout)    # Armazenamento da figura e de seus rectivos dados em fig0
fig1.update_layout(plot_bgcolor="#242423")                   # Definição de fundo 
fig1.update_layout(paper_bgcolor="#242423")                  
fig1.update_layout(font_color="white")                       # Cor da legenda 

In [7]:
#categorias_faturamento[0:5]

In [8]:
'''Gráfico satisfação'''
satisfação = []                                          # Criação de uma lista vazia para receber as satisfações 
coluna_satifação = 33                                    # Coluna 33 se refere as classificações dos clientes    
for linha in range (len(aux_data)):                      # For ira contar a quantidade de linhas presentes em aux_data
    satisfação.append(aux_data[linha][coluna_satifação]) # Adição a lista satifição todas as satisfações

# Filtrando quantidade de cada satisfação  
classificação_satisfação=[]                         # Criação da lista classificação satifação pra agregar as quantidades de cada tipo de satisfação       
good=bad=very_bad=0                                 # Contadores iniciados a partir do zero com o objetivo de agregar a quantidade individual da satisfação   
for linha in range (len(satisfação)):               # Laço para filtrar satisfação ("satisfeito_com_pedido","problemas_de_entrega","problemas_de_qualidade")
    if satisfação[linha]=='satisfeito_com_pedido':  # Contagem de todos as satisfações que recebem = 'satisfeito_com_pedido'
        good+=1
    if satisfação[linha]=='problemas_de_entrega':   # Contagem de todos as satisfações que recebem = 'problemas_de_entrega'   
        bad+=1
    if satisfação[linha]=='problemas_de_qualidade': # Contagem de todos as satisfações que recebem = 'problemas_de_qualidade'
        very_bad+=1
classificação_satisfação.append(good)        # Adição da contagem "good" na lista classificação 
classificação_satisfação.append(bad)         # Adição da contagem "bad" na lista classificação        
classificação_satisfação.append(very_bad)    # Adição da contagem "very_bad" na lista classificação 

#Tipos de satisfação
tipos_classificação=[]                                 # Lista para receber os tipos de classificação
for linhas in range (len(satisfação)):                 # Lendo as linhas  de satisfação para filtrar 
    if satisfação[linhas] not in tipos_classificação:  # Realizando filtragem, só será incluso na lista caso já nn esteja
        tipos_classificação.append(satisfação[linhas]) 

# Relacionando os tipos de satisfações com o a sua quantidade # Resolvendo Nan

tipos_classificação_relação=[]                       # Criando lista para esta relação
for linhas in range (len(tipos_classificação)):      # Laço que ira relacionar o quantidade de cada satisfação
    tipo=tipos_classificação[linhas]                 # Puxando os tipos de classificações   
    total=classificação_satisfação[linhas]           # Puxando as quantidades    
    tipos_classificação_relação.append([tipo,total]) # Adicionando ambos na lista tipos_classificação, relacionando-os consequentemente

# Criando gráfico

trace2 = go.Pie(labels = tipos_classificação,     # trace2 Armazena informações a respeito do gráfico, bem tipo "go.Pie"
               values = classificação_satisfação
              )

# Armazenando gráfico em uma lista

data2 = [trace2]        # Armazenando dados do trace2 em data2                          

# Criando Layout

layout = go.Layout(title='Classificação de Clientes sobre Pedidos')

# Criando figura que será exibida
fig2 = go.Figure(data=data2, layout=layout)
fig2.update_layout(plot_bgcolor="#242423")
fig2.update_layout(paper_bgcolor="#242423")
fig2.update_layout(font_color="white")                       # Cor da legenda 

In [9]:
''''Gráfico Frete_Produto'''

# valor_produto
custo_total=[]                                                               # Lista para adição dos custos totais ("valor*quantidade")
for linha in range (len(aux_data)):                                          # Leitura da matrix (aux_data) # valor coluna 3
    custo_total.append(aux_data[linha][valor])                           # Definindo custo total  

# valor frete
envio=[]                                 # Lista com valor do frete 
frete=4                                  # Agregando 4 a frete por ser sua respectiva coluna
for linha in range (len(aux_data)):      # Leitura da matrix (aux_data)    
    envio.append(aux_data[linha][frete]) # Puxando o valor de cada frete

#relação_valor_frete_produto
frete_produto=[]                     # Criando lista para para relação de ambos, "não é necessário para a plotagem do gáfico, apenas para conferencia de dados"
for linha in range (len(aux_data)):     # Leitura da matrix (aux_data)
    coust = custo_total[linha]          # Agregando valor total a variável 
    frt = envio[linha]                  # Agregando custo de envio a variável
    frete_produto.append([coust,frt])   # Relacionando os dados para a conferência


# Criando gráfico
trace3 = go.Scatter(x = envio,                      # trace2 Armazena informações a respeito do gráfico, bem tipo "go.Scatter"
                   y = custo_total,
                   mode = 'markers',
                   marker = {'color':'#13D6C0'}
                  )
# Armazenando gráfico em uma lista
data3 = [trace3]                                   # Armazenando dados do trace3 em data3 

# Criando Layout
layout = go.Layout(title='Valor de Frete x Valor de Produto',
                   yaxis={'title':'Valor do Produto'},
                   xaxis={'title': 'Valor do Frete'})

# Criando figura que será exibida
fig3 = go.Figure(data=data3, layout=layout)
fig3.update_layout(plot_bgcolor="#242423")
fig3.update_layout(paper_bgcolor="#242423")
fig3.update_layout(font_color="white")                       # Cor da legenda

In [10]:
# hello word

In [11]:
app = dash.Dash(__name__)    # Criação da dash--> abertura da pagina                                    

opcoes = {                                             # Biblioteca opções 
    'Barras': ['Média', 'Trimestre', 'Semestre'],      # Lista de opções para barras 
    'Linhas': []                                       # Lista opções para linhas 
}

app.layout = html.Div(children=[                                            # Div realiza a divisão da pagina 
    html.H1(children= 'ANÁLISE DE DADOS E-COMMERCE', style = {"text-align":"center"}),  # Na primeira termos a atribuição do título (H1) e sua características
    
    

    html.Div([                                                              # Segunda DIV para separar o drop do título
        dcc.Dropdown(                                                       # Caracteríticas do drop
            id='drop',                                                      # id será a indentidade do drop
            options=[                                                       # Filtros fornecidas pelo drop 
                {'label': x, 'value': x} for x in opcoes.keys()],           # De acordo com a opção selecionado, o for irá atribuir este valor a label e value, sendo elas as 
            value = 'Barras', style={'color':'black'}                       # chaves do dicionário 'Barras' 'Linhas' # Valor a princípio será de Barras
        ),
        dcc.RadioItems(id='valores' )],style ={'width':'40%','color':'white'}),  # Determina as caracteristicas do checkbox 'Média', 'Trimestre', 'Semestre'   
      

    html.Hr(), # Divisão por linha física

    html.Div([ # DIV AS DIVISÕES DA DASH.
        dcc.Graph(id='g0'), #grafico0  
        
        dcc.Graph(id= 'g1')],style={'display':'flex'}),   #grafico1    

    html.Hr(),       # Linha de divisão

    html.Div([                                                                      # Divisão real entre o primeiro drop e os gráficos
        dcc.Dropdown(id= 'classe',                                                  # id será a indentidade do drop_classe 
        options=[                                                                   # Filtros fornecidas pelo drop 
            {'label':'cama_mesa_banho','value':'cama_mesa_banho'},
            {'label':'informatica_acessorios','value':'informatica_acessorios'},
            {'label':'moveis_decoracao ','value':'moveis_decoracao'},
            {'label':'relogios_presentes ','value':'relogios_presentes'},
            {'label':'beleza_saude ','value':'beleza_saude'},
            {'label':'esporte_lazer ','value':'esporte_lazer'},
            {'label':'utilidades_domesticas','value':'utilidades_domesticas'},
            {'label':'consoles_games','value':'consoles_games'}],
            value= 'cama_mesa_banho', ),                                            # Valor a princípio será de cama_mesa_banho    
    ],style={'color':'black', 'width':'40%'}),  # width= quanto de espaço toma da tela           

     # Estilo Do drop  

    
    dcc.Graph(id='fig4'),  # Puxando o gráfico # cama_mesa_banho

    html.Hr(), # Linha de divisão física

    html.Div([                                                                      # Divisão real antes dos ultimos gráficos 
        dcc.Graph(figure=fig2),                                                     # Puxando figura 2 'Satisfação'         

        dcc.Graph(figure=fig3)],style={'display':'flex','width':'100%'}),           # Puxando figura 3 'Valor_frete'
    
], style={'background':'#242423', 'color':'white'})                                 # Definindo cor de fundo 
    

    

# função linhas barras
@app.callback(                  # Chamada de valores tanto para entrada quanto para saída 
    [Output('valores', 'options'),  #saida para id = valores no dash (média, trimestre e semestre)
    Output('g0', 'figure'),     # saida para id = g0 (grafico0)
    Output('g1', 'figure')],    #saida para id = g1 (graafico1)
    [Input('drop', 'value'),    #entrada valores do id=drop (Barras ou Linhas)
    Input ('valores','value')]) #entrada valores do id=valores (Média, Trimestre e Semestre)
def funcao (variavel, filtro):  #funçao que atribui o nome variavel ao 1º input (Barras ou Linhas) e filtro ao 2ºinput(Média, Trimestre e Semestre)
    if variavel == 'Barras':
        if filtro == 'Média':

            #laço soma valores 
            total = 0
            valor = 3 
            qnt = 5                              
            for linha in range(len(vendas_periodo)):
                total += aux_data[linha][valor]*aux_data[linha][qnt]     # Faturamento total 

            
            media_periodo = total/len(tempo)                             # Faturamento pela quantidade de tempo
            media_cate= total/len(categorias)                            # Faturamento pela quantidade de categorias        

            cor=[]                                  # Lista vazia para definir cores 
            for x in preço:                         # Se faturamento for menor que a média, a coluna ficara vermelha
                if x < media_periodo:
                    cor.append('red')
                else:                               # Caso contrário, ficara verde 
                    cor.append('#13D6C0')
        

            trace00 = [go.Bar(x=tempo,              # Gráfico ultiliza y como faturamento mensal  
            y=preço,
            marker = {'color': cor,                 # Agrega color de acordo com a lista cores
                        'line': {'color': 'black',  # Especificações do contorno 
                                'width': 3}
                        },
            )]

            media1=[]                               # Lista "meses" com média respectiva 
            for x in range(len(tempo)):             # repetindo 15x (qnt meses) total da média
                media1.append(media_periodo)

            media2=[]                               # Lista "categoria" com média respectiva
            for x in range(len(categorias)):        # repetindo 20x (qnt categorias) total da média    
                media2.append(media_cate)

            data00 = trace00   # Ultilizando gráfico acima (trace00), para plotar um gráfico com a média 

            configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                    yaxis={'title':'Valores em Vendas'},
                                    xaxis={'title':'Periodo'})
            
            fig00 = go.Figure(data=trace00, layout=configuracoes_layout)  ############################ Onde esta trace00 era data00 ###############################################

            fig00.add_trace(go.Line(y=media1, ################### Onde esta Line era Scatter ################################
                                        x= tempo, 
                                        mode='lines',                                                   # mode de marcação 
                                        marker={'color':'red', 'line':{'color': 'black','width': 3}     # estilo da linha 
                                                }
                                        )
                            )  

            fig00.update_layout(plot_bgcolor="#242423")  # background grafico
            fig00.update_layout(paper_bgcolor="#242423") # background grafico
            fig00.update_layout(font_color="white")      # Cor da legenda 


            '''--------'''

            '''a = 0
            for x in range(len(faturamento_total_por_categoria)):     soma para testar o total de faturamento por categoria 
                a += faturamento_total_por_categoria[x]'''

            cor1=[]                                                # Lista de cores para categoria 
            for x in faturamento_total_por_categoria:              # Se x for menor que a média será adiconado vermelho a lista 
                if x < media_cate:
                    cor1.append('red')
                else:                                              # Caso não verde
                    cor1.append('#13D6C0')  


            trace1 = go.Bar(y=faturamento_total_por_categoria,     # plotagem do gráfico de faturamento c=por categoria, 
            x=categorias,
            marker = {'color': cor1},      # relacionando suas respectivas cores                     
            orientation='v'                # Determina a orientação da barra 
                )
            data1 = [trace1]                

            # Criando Layout
            configuracoes_layout = go.Layout(title='Vendas por categoria de Produtos',
                            xaxis=dict(titlefont=dict(size=50,color='black'),tickangle=55),
                            yaxis={'title': 'Faturamento'},
                            
                            )

            # Objeto figura

            fig11 = go.Figure(data=trace1, layout=configuracoes_layout)  ####### trace1 = data1 #######
            fig11.update_layout(plot_bgcolor="#242423")
            fig11.update_layout(paper_bgcolor="#242423")
            fig11.update_layout(font_color="white")                       # Cor da legenda 

            fig11.add_trace(go.Scatter(y=media2,x= categorias, mode='lines', marker={'color':'red', 'line':{'color': 'black','width': 3}})) #Adiciona a linha de media no grafico

            return [{'label': i, 'value': i} for i in opcoes[variavel]], fig00, fig11   #Retorna da função "funcao" conforme os outputs nos id's (valores, g0, g1)

    
        if filtro == 'Trimestre':   # se o filtro escolhido for trimestre, executará assim
            trimestre=[]
            valor_trimestre=[]
            for c in range (0,len(tempo),3):  # esse parâmetro define que começa no mês 0 e vai pular a cada 3 casas
                trimestre.append(tempo[c]) 

            primeiro_trimestre=[]
            segundo_trimestre=[]
            terceiro_trimestre=[]
            quarto_trimestre=[]
            quinto_trimestre=[] 

            for c in range (0,3):     # para o primeiro trimestre  conta a partir do mês zero e pula de 3 em 3
                primeiro_trimestre.append(tempo[c])

            for c in range (3,6):      # para o segundo trimestre  conta a partir do mês 3 e pula de 3 em 3
                segundo_trimestre.append(tempo[c])

            for c in range (6,9):   # para o terceiro trimestre  conta a partir do mês 6 e pula de 3 em 3
                terceiro_trimestre.append(tempo[c])

            for c in range (9,12):   # para o quarto trimestre  conta a partir do mês 9 e pula de 3 em 3
                quarto_trimestre.append(tempo[c])
                
            for c in range (12,15):    # para o quinto trimestre  conta a partir do mês 12 e pula de 3 em 3
                quinto_trimestre.append(tempo[c])

            ''' Valor Primeiro Trimestre '''

            soma=0

            for c in range (len(primeiro_trimestre)):    # percorre a lista e caso os meses sejam iguais aos do 1ºtrimestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if primeiro_trimestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
            
                    

            valor_trimestre.append(y)

            ''' Valor Segundo Trimestre '''

            soma=0

            for c in range (len(segundo_trimestre)):      # percorre a lista e caso os meses sejam iguais aos do 2ºtrimestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if segundo_trimestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_trimestre.append(y)

            ''' Valor Terceiro Trimestre '''

            soma=0

            for c in range (len(terceiro_trimestre)):   # percorre a lista e caso os meses sejam iguais aos do 3ºtrimestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if terceiro_trimestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_trimestre.append(y)


            ''' Valor Quarto Trimestre '''

            soma=0

            for c in range (len(quarto_trimestre)):   # percorre a lista e caso os meses sejam iguais aos do 4º trimestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if quarto_trimestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_trimestre.append(y)


            ''' Valor Quinto Trimestre '''

            soma=0

            for c in range (len(quinto_trimestre)):     # percorre a lista e caso os meses sejam iguais aos do 4ºtrimestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if quinto_trimestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_trimestre.append(y)


            periodo = [1,2,3,4,5]    ########### 
            data0 = [go.Bar(x=periodo,
            y=valor_trimestre,
            marker = {'color': '#13D6C0',
                        'line': {'color': 'black',
                                'width': 3}
                        },
            )]
            configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                    yaxis={'title':'Valores em Vendas'},
                                    xaxis={'title':'Periodo'})

            fig00 = go.Figure(data=data0, layout=configuracoes_layout)
            fig00.update_layout(plot_bgcolor="#242423") #background grafico
            fig00.update_layout(paper_bgcolor="#242423") #background grafico
            fig00.update_layout(font_color="white")                       # Cor da legenda 


            return [{'label': i, 'value': i} for i in opcoes[variavel]], fig00, fig1    #Retorna conforme os outputs nos id's (valores, g0, g1)

        if filtro =='Semestre':   # esse parâmetro define que começa no mês 0 e vai pular a cada 6 casas
            semestre=[]
            valor_semestre=[]
            for c in range (0,len(tempo),6):
                semestre.append(tempo[c])


            primeiro_semestre=[]
            segundo_semestre=[]
            terceiro_semestre=[]

            for c in range (0,6):   # para o primeiro semestre conta a partir do mês zero, pula de 6 em 6 e salva na lista os meses
                primeiro_semestre.append(tempo[c])

            for c in range (6,12):   # para o segundo semestre conta a partir do mês zero, pula de 6 em 6 e salva na lista os meses
                segundo_semestre.append(tempo[c])

            for c in range (12,len(tempo)):  # para o terceiro semestre conta a partir do mês zero, pula de 6 em 6 e salva na lista os meses
                terceiro_semestre.append(tempo[c])

            ''' Valor Primeiro Semestre '''

            soma=0

            for c in range (len(primeiro_semestre)): # percorre a lista e caso os meses sejam iguais aos do 1ºsemestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if primeiro_semestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_semestre.append(y)

            ''' Valor Segundo Semestre '''

            soma=0

            for c in range (len(primeiro_semestre)):      # percorre a lista e caso os meses sejam iguais aos do 2ºsemestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if segundo_semestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_semestre.append(y)

            ''' Valor Terceiro Semestre '''

            soma=0

            for c in range (len(terceiro_semestre)):   # percorre a lista e caso os meses sejam iguais aos do 3ºsemestre, somam-se a variavel soma
                for x in range (len(tempo)):
                    if terceiro_semestre[c]==tempo[x]:
                        soma+=preço[x]
                        
                    y=soma
                    

            valor_semestre.append(y)

            periodo = [1,2,3]       ############## checar
            data0 = [go.Bar(x=periodo,
            y=valor_semestre,
            marker = {'color': '#13D6C0',
                        'line': {'color': 'black',
                                'width': 3}
                        },
            )]
            configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                    yaxis={'title':'Valores em Vendas'},
                                    xaxis={'title':'Periodo'})

            fig00 = go.Figure(data=data0, layout=configuracoes_layout)
            fig00.update_layout(plot_bgcolor="#242423") #background grafico
            fig00.update_layout(paper_bgcolor="#242423") #background grafico
            fig00.update_layout(font_color="white")                       # Cor da legenda 

            return [{'label': i, 'value': i} for i in opcoes[variavel]], fig00, fig1    #Retorna conforme os outputs nos id's (valores, g0, g1)

        return [{'label': i, 'value': i} for i in opcoes[variavel]], fig0, fig1   #Retorna conforme os outputs nos id's (valores, g0, g1) || Retorno do if variavel == 'Barras'

    if variavel == 'Linhas':    # caso seja selecionada a opção linhas do dash, roda a rotina abaixo

        data00 = [go.Line(x=tempo,
           y=preço,
           marker = {'color': '#13D6C0',
                     'line': {'color': 'black',
                              'width': 3}
                    },
          )]
        configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                 yaxis={'title':'Valores em Vendas'},
                                 xaxis={'title':'Periodo'})

        fig00 = go.Figure(data=data00, layout=configuracoes_layout)
        fig00.update_layout(plot_bgcolor="#242423") #background grafico
        fig00.update_layout(paper_bgcolor="#242423") #background grafico
        fig00.update_layout(font_color="white")                       # Cor da legenda 



        trace1 = go.Line(y=faturamento_total_por_categoria,
                x=categorias,
                marker = {'color': '#13D6C0'},
                orientation='v'
              )

        data1 = [trace1]

        # Criando Layout
        configuracoes_layout = go.Layout(title='Vendas por categoria de Produtos',
                        xaxis=dict(titlefont=dict(size=50,color='black'),tickangle=55),
                        yaxis={'title': 'Quantidade vendida'},
                        
                        ) 

        # Objeto figura

        fig11 = go.Figure(data=data1, layout=configuracoes_layout)
        fig11.update_layout(plot_bgcolor="#242423")
        fig11.update_layout(paper_bgcolor="#242423")
        fig11.update_layout(font_color="white")                       # Cor da legenda 

        
        return [{'label': i, 'value': i} for i in opcoes[variavel]], fig00, fig11   #Retorna conforme os outputs nos id's (valores, g0, g1)

    
    
    else:
        return [{'label': i, 'value': i} for i in opcoes[variavel]], fig0, fig1     #Retorna conforme os outputs nos id's (valores, g0, g1)

@app.callback(
    Output('fig4', 'figure'),   # grafico tipo de produtos # cama_mesa_banho
    [Input('classe','value')]
)
def faz (variavel):
    faturamento_tempo_prod = [] 
    time = []
    value = []
    for x in range(len(aux_data)):
        faturamento_tempo_prod.append([aux_data[x][34],aux_data[x][3]*aux_data[x][5]]) ##### checar ##### coluna 34 = tempo , coluna 5 = quantidade e coluna 3 = valor, coluna 14 = categoria #
        if aux_data[x][14] == variavel:    # se o item escolhido pelo usuário for igual ao item, ele adiciona a data do item vendido a lista "time"
            if aux_data[x][34] not in time:
                time.append(aux_data[x][34])

    time.sort()
    soma= 0 

    for y in range(len(time)):
        for x in range(len(faturamento_tempo_prod)):

            if faturamento_tempo_prod[x][0] == time[y] and aux_data[x][14]== variavel:  # se o tempo escolhido pelo usuário baterem com o tempo da venda e da lista fat_tempo_prod, ele soma
                soma+= faturamento_tempo_prod[x][1]
        value.append(soma)
        soma = 0

    data00 = [go.Bar(x=time,                             # data0 Armazena informações a respeito do gráfico, bem tipo "go.bar"                           
            y=value,                             # A correspondenciado dos eixos, "x=time" e "y=time"                marker = {'color': '#13D6C0',        # maker=recebe caracterísca de layout da barra, sendo color sua cor, line sendo linha, com sua respectivas
            marker={'line': {'color': 'black', # caracteristicas como cor e espessura "width"
                            'width': 3},
                            'color':'#13D6C0'
                    },
            )
    ] 


    # Criando Layout                                                    
    configuracoes_layout = go.Layout(title= variavel,             # Definição da legenda das linhas como título
                                    yaxis={'title':'Valores em Vendas'},   # Definição da legenda do eixo y e x    
                                    xaxis={'title':'Periodo'})

    # Objeto figura

    fig4 = go.Figure(data=data00, layout=configuracoes_layout)  # Armazenamento da figura e de seus rectivos dados em fig0         
    fig4.update_layout(plot_bgcolor="#242423")                 # Definição de fundo 
    fig4.update_layout(paper_bgcolor="#242423")  
    fig4.update_layout(font_color="white")  
        
    return fig4


if __name__== '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Dec/2020 06:37:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 06:37:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 06:37:50] "GET /_favicon.ico?v=1.17.0 HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 06:37:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 06:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2020 06:37:50] "POST /_dash-update-component HTTP/1.1" 200 -
